In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from sklearn import preprocessing

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
##模型训练

In [3]:
sample = np.loadtxt('train.dat')
x = sample[:,1:-1]
scaler=preprocessing.StandardScaler().fit(x)
x=scaler.transform(x)
y = sample[:,-1]
x_train,y_train=x,y

In [5]:

model = MLPClassifier(solver='adam',max_iter=300,learning_rate_init=0.001,hidden_layer_sizes=(64,32,32,16,16))
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64, 32, 32, 16, 16), learning_rate='constant',
       learning_rate_init=0.001, max_iter=300, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [44]:
##训练集效果评估
y_pred =  model.predict(x_train)
print(classification_report(y_train,y_pred))

             precision    recall  f1-score   support

        0.0       0.91      0.90      0.90      3160
        1.0       0.90      0.91      0.90      3120

avg / total       0.90      0.90      0.90      6280



In [26]:
##测试集效果评估
y_pred =  model.predict(x_test)
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.89      0.75      0.81       657
        1.0       0.77      0.90      0.83       599

avg / total       0.83      0.82      0.82      1256



In [48]:
##保存模型
joblib.dump(model,'../system/model.pkl')

['../system/model_282.pkl']

In [13]:
##读入模型
model = joblib.load('model.pkl')

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


##微博排序##

In [14]:
def load_uid():
    uid_list = []
    with open('../../data/uid.txt','r',encoding = 'utf-8') as f1:
        while True:
            line = f1.readline()
            if not line:
                break
            uid_list.append(line.strip())
        f1.close()
    return uid_list

In [15]:
def load_test():
    user_test = []
    original = {}
    filename = '../../data/test/'+str(uid)+'.dat'
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = eval(line)
            wid = line['id']
            userid = line['userid']
            originalid = line['original']
            if 'id' not in originalid:
                original[wid] = wid
            else:
                original[wid] = originalid['id']
            if userid == uid:
                user_test.append(original[wid])
        f.close()
    return user_test

In [16]:
uid_list = load_uid()
for num in [10,20,30,40,50]:
    hit_count = 0
    for uid in uid_list:
        uid = int(uid)
        filename = './test/'+str(uid)+'.dat'
        test = np.loadtxt(filename)
        wid = test[:,0]
        x = test[:,1:]
        x = scaler.transform(x)
        pre = model.predict_proba(x)
        result = {}
        for i in range(0,len(wid)):
            result[wid[i]] = pre[i][1]
        result = sorted(result.items(),key=lambda x:x[1],reverse=True)
        user_test = load_test()
        count = 0
        for i in range(0,num):
            wid = int(result[i][0])
            wid = str(wid)
            if wid in user_test:
                count += 1
        hit_count += count
    print("top{},recall: {}".format(num,hit_count/800)) 
    print("top{},precision: {}".format(num,hit_count/(num*20)))

top10,recall: 0.135
top10,precision: 0.54
top20,recall: 0.21375
top20,precision: 0.4275
top30,recall: 0.26375
top30,precision: 0.3516666666666667
top40,recall: 0.30375
top40,precision: 0.30375
top50,recall: 0.34625
top50,precision: 0.277
